In [5]:
!python3 -m pip install -r requirements.txt

In [7]:
from dotenv import dotenv_values

import pandas as pd

config = dotenv_values(".env")
df = pd.read_csv(str(config["WOS_DATA"]), sep="\t")

In [10]:
df.head(5)

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,WC,WE,SC,GA,PM,OA,HC,HP,DA,UT
0,J,"Zhang, H; Zhang, ZY; Zhang, L; Yang, YF; Kang,...",NaN,NaN,NaN,"Zhang, Hong; Zhang, Zeyu; Zhang, Lei; Yang, Yi...",NaN,NaN,Object Tracking for a Smart City Using IoT and...,SENSORS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,J,"Yar, H; Imran, AS; Khan, ZA; Sajjad, M; Kastra...",NaN,NaN,NaN,"Yar, Hikmat; Imran, Ali Shariq; Khan, Zulfiqar...",NaN,NaN,Towards Smart Home Automation Using IoT-Enable...,SENSORS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,J,"Gheisari, M; Wang, GJ; Chen, SH",NaN,NaN,NaN,"Gheisari, Mehdi; Wang, Guojun; Chen, Shuhong",NaN,NaN,An Edge Computing-enhanced Internet of Things ...,COMPUTERS & ELECTRICAL ENGINEERING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,J,"Hossain, SKA; Rahman, MA; Hossain, MA",NaN,NaN,NaN,"Hossain, S. K. Alamgir; Rahman, Md Anisur; Hos...",NaN,NaN,Edge computing framework for enabling situatio...,JOURNAL OF PARALLEL AND DISTRIBUTED COMPUTING,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,J,"Wu, TY; Kong, FF; Wang, LY; Chen, YC; Kumari, ...",NaN,NaN,NaN,"Wu, Tsu-Yang; Kong, Fangfang; Wang, Liyang; Ch...",NaN,NaN,Toward Smart Home Authentication Using PUF and...,SENSORS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


authors addresses:

In [27]:
df = df[df["C1"].notna()]
df = df.copy()

In [34]:
def extract_main_author_address(address):
    if isinstance(address, str):
        parts = address.split("; [")
        if len(parts) > 1:
            return parts[0].split("] ")[1] if "] " in parts[0] else parts[0]
    return ""
df["main_author_address"] = df["C1"].apply(extract_main_author_address)

In [ ]:
def extract_country(address):
  if isinstance(address, str):
    # Special case for USA at the end with state and zip
    if address.strip().endswith("USA"):
      return "USA"
    parts = address.split(",")
    return parts[-1].strip() if len(parts) > 1 else ""
  return ""

df["country"] = df["main_author_address"].apply(extract_country)

In [44]:
# show unique countries and their occurrences
country_counts = df["country"].value_counts()
print(country_counts)

country
Peoples R China    429
                   201
India               50
South Korea         40
USA                 35
Saudi Arabia        25
Taiwan              17
England             16
Australia           16
Canada              15
Pakistan            12
Spain               11
Vietnam             10
Malaysia            10
Poland               8
Italy                7
Iran                 7
Brazil               7
Finland              7
Greece               5
Algeria              5
Japan                5
Jordan               3
Turkiye              3
France               3
South Africa         3
Bangladesh           3
Turkey               3
Iraq                 3
Ireland              3
Singapore            3
Norway               2
Portugal             2
Germany              2
Morocco              2
Denmark              2
Egypt                2
Romania              2
Russia               2
Austria              1
Colombia             1
Thailand             1
Tunisia              1
Cze